# Limpieza y transformación de datos

Este notebook aplica la limpieza y transformación necesarias al dataset de accidentes viales para estandarizar su estructura y facilitar el análisis posterior.

## Objetivo del notebook

Estandarizar y depurar el dataset para prepararlo para análisis visual y espacial.  
Las operaciones clave incluyen:

- Conversión de tipos:
  - Fechas → `datetime`
  - Horas → `datetime.time`
  - Coordenadas → `float`
  - Texto categórico → `str` en minúsculas
  - Booleanos → `bool`
- Limpieza estructural:
  - Eliminación de duplicados
  - Eliminación de columnas vacías o con un solo valor
  - Eliminación de filas con valores nulos en campos clave (`latitud`, `fecha_captura`)
 

Estas transformaciones permiten un análisis más confiable y evitan errores en las siguientes etapas del proyecto.

---

## Tipos de datos en Pandas utilizados

| Tipo de dato        | Descripción                                                  |
|---------------------|--------------------------------------------------------------|
| `object`            | Tipo genérico, usualmente texto (strings).                   |
| `int64`             | Números enteros (sin decimales).                             |
| `float64`           | Números decimales (punto flotante).                          |
| `datetime64[ns]`    | Fecha y hora (datetime).                                     |
| `bool`              | Valores lógicos: `True` o `False`.                           |
| `category`          | Texto optimizado para variables categóricas repetitivas.     |

---

  El resultado es un archivo limpio y confiable, guardado como  
`data/accidentes_cdmx_limpio.csv`, listo para análisis exploratorio y visualización en el dashboard.


In [ ]:
import pandas as pd  # Cargamos pandas

# Leemos el CSV original
df = pd.read_csv('../data/accidentes_cdmx.csv')

# Revisamos los tipos de dato por columna
print("\nTipos de datos por columna:")
print(df.dtypes)

# Como pandas no convierte automáticamente las columnas de hora a tipo datetime, lo hacemos manualmente
print("hora_evento:", type(df['hora_evento'].iloc[0]))



In [ ]:

# Conversión de columnas que requieren cambio de tipo

# Conversión de fechas
# Convierte 'fecha_evento' a tipo datetime usando el formato día/mes/año
df['fecha_evento'] = pd.to_datetime(df['fecha_evento'], format='%d/%m/%Y', errors='coerce')

# Convierte 'fecha_captura' a datetime con el mismo formato
df['fecha_captura'] = pd.to_datetime(df['fecha_captura'], format='%d/%m/%Y', errors='coerce')

# Limpieza y conversión de hora
# Asegura que la columna 'hora_evento' sea texto limpio (sin espacios ni valores 'nan')
df['hora_evento'] = df['hora_evento'].astype(str).str.strip().replace('nan', pd.NA)

# Convierte 'hora_evento' a tipo datetime solo con hora, usando el formato 24h
df['hora_evento'] = pd.to_datetime(df['hora_evento'], format='%H:%M', errors='coerce').dt.time

# Conversión de coordenadas
# Convierte 'latitud' a tipo numérico (float), forzando errores como NaN
df['latitud'] = pd.to_numeric(df['latitud'], errors='coerce')

# Convierte 'longitud' a tipo numérico también, para evitar errores futuros
df['longitud'] = pd.to_numeric(df['longitud'], errors='coerce')  

# Si existe la columna, la limpiamos y convertimos a tipo booleano
if 'trasladado_lesionados' in df.columns:
    df['trasladado_lesionados'] = (
        df['trasladado_lesionados']
        .astype(str)         # Aseguramos que sea texto
        .str.strip()         # Quitamos espacios
        .str.lower()         # Pasamos a minúsculas
        .map({'si': True, 'no': False})  # Convertimos 'si'/'no' a booleanos
        .fillna(False)       # Rellenamos nulos como False
        .astype(bool)        # Aseguramos el tipo bool final
    )


# Estandarización de textos en matrícula médica 
# Si existe la columna 'matricula_unidad_medica':
if 'matricula_unidad_medica' in df.columns:
    # Convierte a string, elimina espacios y pasa todo a mayúsculas
    df['matricula_unidad_medica'] = df['matricula_unidad_medica'].astype(str).str.strip().str.upper()

# Limpieza de columnas categóricas de texto
# Lista de columnas que deben estandarizarse como texto uniforme (en minúsculas, sin espacios)
columnas_texto = [
    'tipo_evento', 'folio', 'punto_1', 'punto_2', 'colonia', 'alcaldia', 'sector',
    'unidad_a_cargo', 'tipo_de_interseccion', 'clasificacion_de_la_vialidad',
    'sentido_de_circulacion', 'dia', 'prioridad', 'origen', 'unidad_medica_de_apoyo'
]

# Para cada columna en la lista:
for col in columnas_texto:
    if col in df.columns:
        # Asegura que sea texto limpio (sin espacios, en minúsculas)
        df[col] = df[col].astype(str).str.strip().str.lower()





In [ ]:
# Comprobacion
# Revisamos los tipos de dato por columna
print("\nTipos de datos por columna:")
print(df.dtypes)

# Como pandas no convierte automáticamente las columnas de hora a tipo datetime, lo hacemos manualmente
print("hora_evento:", type(df['hora_evento'].iloc[0]))

In [ ]:

import os
os.makedirs("data", exist_ok=True)

# Revision

# Valores nulos por columna
nulos_totales = df.isnull().sum()
porcentaje_nulos = (nulos_totales / len(df)) * 100
print("Nulos por columna:")
print(pd.DataFrame({'Nulos': nulos_totales, '%': porcentaje_nulos}).sort_values(by='%', ascending=False))

# Filas duplicadas
duplicados = df.duplicated().sum()
print(f"Filas duplicadas: {duplicados}")

# Columnas vacías o constantes
columnas_vacias = df.columns[df.isnull().sum() == len(df)]
columnas_constantes = [col for col in df.columns if df[col].nunique() == 1]
print(f"Columnas completamente vacías: {list(columnas_vacias)}")
print(f"Columnas con un solo valor: {columnas_constantes}")



In [ ]:
# 3. LIMPIEZA DE DATOS

# Eliminar duplicados
df = df.drop_duplicates()

# Eliminar columnas vacías y constantes
df = df.drop(columns=columnas_vacias)
df = df.drop(columns=columnas_constantes)

# Eliminar filas con nulos clave
df = df.dropna(subset=['latitud', 'fecha_captura'])

In [ ]:
# Comprobacion

# Valores nulos por columna
nulos_totales = df.isnull().sum()
porcentaje_nulos = (nulos_totales / len(df)) * 100
print("Nulos por columna:")
print(pd.DataFrame({'Nulos': nulos_totales, '%': porcentaje_nulos}).sort_values(by='%', ascending=False))

# Filas duplicadas
duplicados = df.duplicated().sum()
print(f"Filas duplicadas: {duplicados}")

# Columnas vacías o constantes
columnas_vacias = df.columns[df.isnull().sum() == len(df)]
columnas_constantes = [col for col in df.columns if df[col].nunique() == 1]
print(f"Columnas completamente vacías: {list(columnas_vacias)}")
print(f"Columnas con un solo valor: {columnas_constantes}")

In [ ]:
# GUARDAmos DEL DATASET LIMPIO
df.to_csv('../data/accidentes_cdmx_limpio.csv', index=False, encoding='utf-8')
print("\nDataset limpio guardado en data/accidentes_cdmx_limpio.csv")
